In [1]:
import os
import pandas as pd
import time
import collections
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# initialize the initial learning rate, number of epochs and batch size
INIT_LR = 1e-3
EPOCHS = 20
BS = 16
# Data directory and data category 
DIRECTORY = r"D:/Datasets/BGYF_Dataset_processed"
CATEGORIES = os.listdir(DIRECTORY)
CATEGORY_SIZE = len(CATEGORIES)

data = []
labels = []


In [3]:
data = []
labels = []
# get all the data and labels in given directory
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    filelist = os.listdir(path)
    for fichier in filelist[:]: # filelist[:] makes a copy of filelist.
        if not(fichier.endswith(".jpg")):
            filelist.remove(fichier)
    
    for img in filelist:
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)


In [4]:
# check the data count and balance  
counter=collections.Counter(labels)
counter

Counter({'E': 90, 'H': 110})

In [5]:
# perform one-hot encoding on the labels
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example
values = array(labels)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
labels = onehot_encoder.fit_transform(integer_encoded)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.30, stratify=labels, random_state=42)

In [6]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [7]:
names = ["VGG16"]
optimizers = ["Adam","SGD","RMSprop","Adadelta","Adagrad","Adamax","Nadam","Ftrl"]
loss_funcs = []
i=0
for opti in optimizers:
    NAME = "{}-{}".format(opti,int(time.time()))
    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

    baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

    # construct the head of the model that will be placed on top of the
    # the base model

    
    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
#    headModel = Dense(1024, activation="relu")(headModel)
#    headModel = Dropout(0.2)(headModel)
    headModel = Dense(CATEGORY_SIZE, activation="sigmoid")(headModel)

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    model = Model(inputs=baseModel.input, outputs=headModel)

    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    layer_num = 0
    for layer in baseModel.layers:
        layer_num = layer_num+1
        if layer_num <17:
            layer.trainable = False
        else:
            layer.trainable = True
        
    # compile our model
    if opti == "Adam":
        print("Adam")
        opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    elif opti == "SGD":
        print("SGD")
        opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    elif opti == "RMSprop":
        opt = RMSprop(lr=INIT_LR, decay=INIT_LR / EPOCHS)
        print("RMSprop")
    elif opti == "Adadelta":
        opt = Adadelta(lr=INIT_LR, decay=INIT_LR / EPOCHS)
        print("Adadelta")
    elif opti == "Adagrad":
        opt = Adagrad(lr=INIT_LR, decay=INIT_LR / EPOCHS)
        print("Adagrad")
    elif opti == "Adamax":
        opt = Adamax(lr=INIT_LR, decay=INIT_LR / EPOCHS)
        print("Adamax")
    elif opti == "Nadam":
        opt = Nadam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
        print("Nadam")
    elif opti == "Ftrl":
        opt = Ftrl(lr=INIT_LR, decay=INIT_LR / EPOCHS)
        print("Ftrl")
        
    model.compile(loss="binary_crossentropy", optimizer=opt,
        metrics=["accuracy"])

#     model.fit(trainX, trainY,
#         batch_size=BS,
#         epochs=EPOCHS,
#         validation_data=(testX, testY),
#         callbacks=[tensorboard])       

        #model.summary()    

        # train the head of the network
    model.fit(
        aug.flow(trainX, trainY, batch_size=BS),
        steps_per_epoch=len(trainX) // BS,
        validation_data=(testX, testY),
        validation_steps=len(testX) // BS,
        epochs=EPOCHS,
        callbacks=[tensorboard])



    # make predictions on the testing setk
    #print("[INFO] evaluating network...")
    #predIdxs = model.predict(testX, batch_size=BS)

    # for each image in the testing set we need to find the index of the
    # label with corresponding largest predicted probability
    #predIdxs = np.argmax(predIdxs, axis=1)

    # show a nicely formatted classification report
    # print(classification_report(testY.argmax(axis=1), predIdxs,
    #       target_names=lb.classes_))

    # serialize the model to disk
    #print("[INFO] saving multiclass model...")
    #try:
    #    model.save("aflotoksin.h5", save_format="h5")
    #except:
    #    pass


Adam


C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
8/8 [==============================] - 19s 2s/step - loss: 0.7791 - accuracy: 0.6613 - val_loss: 0.4269 - val_accuracy: 0.8500
Epoch 2/20
8/8 [==============================] - 8s 1s/step - loss: 0.4745 - accuracy: 0.8226 - val_loss: 0.3534 - val_accuracy: 0.8667
Epoch 3/20
8/8 [==============================] - 8s 1s/step - loss: 0.3287 - accuracy: 0.8790 - val_loss: 0.3666 - val_accuracy: 0.9167
Epoch 4/20
8/8 [==============================] - 8s 1s/step - loss: 0.3358 - accuracy: 0.8871 - val_loss: 0.3459 - val_accuracy: 0.8667
Epoch 5/20
8/8 [==============================] - 8s 1s/step - loss: 0.3188 - accuracy: 0.8871 - val_loss: 0.3528 - val_accuracy: 0.8500
Epoch 6/20
8/8 [==============================] - 8s 1s/step - loss: 0.3298 - accuracy: 0.8790 - val_loss: 0.3578 - val_accuracy: 0.9000
Epoch 7/20
8/8 [==============================] - 8s 1s/step - loss: 0.3164 - accuracy: 0.8790 - val_loss: 0.3511 - val_accuracy: 0.8500
Epoch 8/20
8/8 [========================

C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
8/8 [==============================] - 9s 1s/step - loss: 0.7109 - accuracy: 0.4758 - val_loss: 0.6644 - val_accuracy: 0.5500
Epoch 2/20
8/8 [==============================] - 9s 1s/step - loss: 0.7125 - accuracy: 0.4677 - val_loss: 0.6715 - val_accuracy: 0.4500
Epoch 3/20
8/8 [==============================] - 8s 1s/step - loss: 0.6856 - accuracy: 0.6048 - val_loss: 0.6519 - val_accuracy: 0.6333
Epoch 4/20
8/8 [==============================] - 8s 1s/step - loss: 0.6846 - accuracy: 0.5806 - val_loss: 0.6489 - val_accuracy: 0.8167
Epoch 5/20
8/8 [==============================] - 8s 1s/step - loss: 0.6758 - accuracy: 0.5403 - val_loss: 0.7006 - val_accuracy: 0.4500
Epoch 6/20
8/8 [==============================] - 8s 1s/step - loss: 0.6675 - accuracy: 0.5726 - val_loss: 0.6717 - val_accuracy: 0.5500
Epoch 7/20
8/8 [==============================] - 8s 1s/step - loss: 0.6847 - accuracy: 0.5312 - val_loss: 0.6309 - val_accuracy: 0.6167
Epoch 8/20
8/8 [=========================

...

8/8 [==============================] - 9s 1s/step - loss: 0.7158 - accuracy: 0.6210 - val_loss: 0.3750 - val_accuracy: 0.8333
Epoch 2/20
8/8 [==============================] - 8s 1s/step - loss: 0.4050 - accuracy: 0.8145 - val_loss: 0.4590 - val_accuracy: 0.8667
Epoch 3/20
8/8 [==============================] - 8s 1s/step - loss: 0.4389 - accuracy: 0.8306 - val_loss: 0.2915 - val_accuracy: 0.8667
Epoch 4/20
8/8 [==============================] - 8s 1s/step - loss: 0.3072 - accuracy: 0.8906 - val_loss: 0.2866 - val_accuracy: 0.9167
Epoch 5/20
8/8 [==============================] - 8s 1s/step - loss: 0.3097 - accuracy: 0.8952 - val_loss: 0.3054 - val_accuracy: 0.8667
Epoch 6/20
8/8 [==============================] - 8s 1s/step - loss: 0.3033 - accuracy: 0.8952 - val_loss: 0.2933 - val_accuracy: 0.9000
Epoch 7/20
8/8 [==============================] - 8s 1s/step - loss: 0.2782 - accuracy: 0.9113 - val_loss: 0.3211 - val_accuracy: 0.8500
Epoch 8/20
8/8 [==============================] - 8s

C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\keras\optimizer_v2\nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


Nadam
Epoch 1/20
8/8 [==============================] - 10s 1s/step - loss: 0.6999 - accuracy: 0.7097 - val_loss: 0.3812 - val_accuracy: 0.9167
Epoch 2/20
8/8 [==============================] - 9s 1s/step - loss: 0.3748 - accuracy: 0.8281 - val_loss: 0.3069 - val_accuracy: 0.9333
Epoch 3/20
8/8 [==============================] - 8s 1s/step - loss: 0.3181 - accuracy: 0.9032 - val_loss: 0.2680 - val_accuracy: 0.9000
Epoch 4/20
8/8 [==============================] - 8s 1s/step - loss: 0.3253 - accuracy: 0.8790 - val_loss: 0.2860 - val_accuracy: 0.8833
Epoch 5/20
8/8 [==============================] - 9s 1s/step - loss: 0.3841 - accuracy: 0.8306 - val_loss: 0.3964 - val_accuracy: 0.8333
Epoch 6/20
8/8 [==============================] - 9s 1s/step - loss: 0.3739 - accuracy: 0.8710 - val_loss: 0.2910 - val_accuracy: 0.9000
Epoch 7/20
8/8 [==============================] - 9s 1s/step - loss: 0.2986 - accuracy: 0.8952 - val_loss: 0.3317 - val_accuracy: 0.8500
Epoch 8/20
8/8 [==================

C:\Users\MONSTER\anaconda3\envs\image\lib\site-packages\keras\optimizer_v2\ftrl.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Ftrl, self).__init__(name, **kwargs)


Ftrl
Epoch 1/20
8/8 [==============================] - 9s 1s/step - loss: 0.7195 - accuracy: 0.5161 - val_loss: 0.6930 - val_accuracy: 0.5500
Epoch 2/20
8/8 [==============================] - 8s 1s/step - loss: 0.6930 - accuracy: 0.5484 - val_loss: 0.6930 - val_accuracy: 0.5500
Epoch 3/20
8/8 [==============================] - 8s 1s/step - loss: 0.6931 - accuracy: 0.5161 - val_loss: 0.6930 - val_accuracy: 0.5500
Epoch 4/20
8/8 [==============================] - 8s 1s/step - loss: 0.6930 - accuracy: 0.5323 - val_loss: 0.6930 - val_accuracy: 0.5500
Epoch 5/20
8/8 [==============================] - 8s 1s/step - loss: 0.6930 - accuracy: 0.5323 - val_loss: 0.6930 - val_accuracy: 0.5500
Epoch 6/20
8/8 [==============================] - 8s 1s/step - loss: 0.6930 - accuracy: 0.5391 - val_loss: 0.6929 - val_accuracy: 0.5500
Epoch 7/20
8/8 [==============================] - 8s 1s/step - loss: 0.6930 - accuracy: 0.5323 - val_loss: 0.6929 - val_accuracy: 0.5500
Epoch 8/20
8/8 [====================

In [6]:
import os
import pandas as pd
import time
import collections
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [7]:
# initialize the initial learning rate, number of epochs and batch size
EPOCHS = 200
BS = 16
# Data directory and data category 
DIRECTORY = r"D:/Datasets/BGYF_Dataset_processed"
CATEGORIES = os.listdir(DIRECTORY)
CATEGORY_SIZE = len(CATEGORIES)

data = []
labels = []


In [8]:
data = []
labels = []
# get all the data and labels in given directory
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    filelist = os.listdir(path)
    for fichier in filelist[:]: # filelist[:] makes a copy of filelist.
        if not(fichier.endswith(".jpg")):
            filelist.remove(fichier)
    
    for img in filelist:
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)


In [9]:
# check the data count and balance  
counter=collections.Counter(labels)
counter

Counter({'E': 90, 'H': 110})

In [10]:
# perform one-hot encoding on the labels
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example
values = array(labels)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
labels = onehot_encoder.fit_transform(integer_encoded)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.30, stratify=labels, random_state=42)

In [11]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [13]:
INIT_LR = 1e-5

NAME = "Adamax-{}-{}".format(INIT_LR,int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

baseModel = VGG16(weights="imagenet", include_top=False,
input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(CATEGORY_SIZE, activation="sigmoid")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
layer_num = 0
for layer in baseModel.layers:
    layer_num = layer_num+1
    if layer_num <17:
        layer.trainable = False
    else:
        layer.trainable = True
        
opt = Adamax(lr=INIT_LR, decay=INIT_LR / EPOCHS)
       
        
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

#     model.fit(trainX, trainY,
#         batch_size=BS,
#         epochs=EPOCHS,
#         validation_data=(testX, testY),
#         callbacks=[tensorboard])       

        #model.summary()    

        # train the head of the network
model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS,
    callbacks=[tensorboard])



    # make predictions on the testing setk
    #print("[INFO] evaluating network...")
    #predIdxs = model.predict(testX, batch_size=BS)

    # for each image in the testing set we need to find the index of the
    # label with corresponding largest predicted probability
    #predIdxs = np.argmax(predIdxs, axis=1)

    # show a nicely formatted classification report
    # print(classification_report(testY.argmax(axis=1), predIdxs,
    #       target_names=lb.classes_))

    # serialize the model to disk
    #print("[INFO] saving multiclass model...")
    #try:
    #    model.save("aflotoksin.h5", save_format="h5")
    #except:
    #    pass


Epoch 1/200
8/8 [==============================] - 9s 1s/step - loss: 0.7040 - accuracy: 0.5484 - val_loss: 0.6957 - val_accuracy: 0.5500
Epoch 2/200
8/8 [==============================] - 8s 1s/step - loss: 0.6963 - accuracy: 0.5323 - val_loss: 0.6850 - val_accuracy: 0.5500
Epoch 3/200
8/8 [==============================] - 8s 1s/step - loss: 0.6754 - accuracy: 0.5726 - val_loss: 0.6787 - val_accuracy: 0.5500
Epoch 4/200
8/8 [==============================] - 8s 1s/step - loss: 0.6670 - accuracy: 0.5781 - val_loss: 0.6735 - val_accuracy: 0.5500
Epoch 5/200
8/8 [==============================] - 8s 1s/step - loss: 0.6762 - accuracy: 0.5645 - val_loss: 0.6688 - val_accuracy: 0.5667
Epoch 6/200
8/8 [==============================] - 8s 1s/step - loss: 0.6679 - accuracy: 0.5806 - val_loss: 0.6641 - val_accuracy: 0.5667
Epoch 7/200
8/8 [==============================] - 8s 1s/step - loss: 0.6674 - accuracy: 0.5645 - val_loss: 0.6600 - val_accuracy: 0.5667
Epoch 8/200
8/8 [=================

8/8 [==============================] - 8s 1s/step - loss: 0.4699 - accuracy: 0.8065 - val_loss: 0.3794 - val_accuracy: 0.9000
Epoch 61/200
8/8 [==============================] - 8s 1s/step - loss: 0.4514 - accuracy: 0.8548 - val_loss: 0.3753 - val_accuracy: 0.9000
Epoch 62/200
8/8 [==============================] - 8s 1s/step - loss: 0.4425 - accuracy: 0.8468 - val_loss: 0.3688 - val_accuracy: 0.8833
Epoch 63/200
8/8 [==============================] - 8s 1s/step - loss: 0.4607 - accuracy: 0.8306 - val_loss: 0.3633 - val_accuracy: 0.8833
Epoch 64/200
8/8 [==============================] - 8s 1s/step - loss: 0.4126 - accuracy: 0.8468 - val_loss: 0.3593 - val_accuracy: 0.8833
Epoch 65/200
8/8 [==============================] - 8s 1s/step - loss: 0.4560 - accuracy: 0.8387 - val_loss: 0.3554 - val_accuracy: 0.8833
Epoch 66/200
8/8 [==============================] - 8s 1s/step - loss: 0.4418 - accuracy: 0.8468 - val_loss: 0.3520 - val_accuracy: 0.8833
Epoch 67/200
8/8 [======================

Epoch 119/200
8/8 [==============================] - 8s 1s/step - loss: 0.3165 - accuracy: 0.8871 - val_loss: 0.2547 - val_accuracy: 0.9000
Epoch 120/200
8/8 [==============================] - 8s 1s/step - loss: 0.3357 - accuracy: 0.8710 - val_loss: 0.2540 - val_accuracy: 0.9000
Epoch 121/200
8/8 [==============================] - 8s 1s/step - loss: 0.3150 - accuracy: 0.9113 - val_loss: 0.2537 - val_accuracy: 0.9000
Epoch 122/200
8/8 [==============================] - 8s 1s/step - loss: 0.3412 - accuracy: 0.8548 - val_loss: 0.2539 - val_accuracy: 0.9000
Epoch 123/200
8/8 [==============================] - 8s 1s/step - loss: 0.3363 - accuracy: 0.8750 - val_loss: 0.2532 - val_accuracy: 0.9000
Epoch 124/200
8/8 [==============================] - 8s 1s/step - loss: 0.3334 - accuracy: 0.8548 - val_loss: 0.2545 - val_accuracy: 0.9000
Epoch 125/200
8/8 [==============================] - 8s 1s/step - loss: 0.3125 - accuracy: 0.8828 - val_loss: 0.2536 - val_accuracy: 0.9000
Epoch 126/200
8/8 [=

8/8 [==============================] - 8s 1s/step - loss: 0.2849 - accuracy: 0.8828 - val_loss: 0.2366 - val_accuracy: 0.9167
Epoch 178/200
8/8 [==============================] - 8s 1s/step - loss: 0.2745 - accuracy: 0.9032 - val_loss: 0.2363 - val_accuracy: 0.9167
Epoch 179/200
8/8 [==============================] - 8s 1s/step - loss: 0.3207 - accuracy: 0.8710 - val_loss: 0.2363 - val_accuracy: 0.9000
Epoch 180/200
8/8 [==============================] - 8s 1s/step - loss: 0.2955 - accuracy: 0.8871 - val_loss: 0.2361 - val_accuracy: 0.9000
Epoch 181/200
8/8 [==============================] - 8s 1s/step - loss: 0.2711 - accuracy: 0.9194 - val_loss: 0.2354 - val_accuracy: 0.9000
Epoch 182/200
8/8 [==============================] - 8s 1s/step - loss: 0.2959 - accuracy: 0.8871 - val_loss: 0.2350 - val_accuracy: 0.9000
Epoch 183/200
8/8 [==============================] - 8s 1s/step - loss: 0.2981 - accuracy: 0.9032 - val_loss: 0.2347 - val_accuracy: 0.9000
Epoch 184/200
8/8 [===============